In [1]:
import os
import gc
import re
import sys
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

## 一.在售车配置信息提取

In [156]:
def delete_useless_char(df,column_name):
    text = df[column_name]
    if str(text) == 'nan':
        return df[column_name]
    text = text.replace('&nbsp','')
    text = text.replace(';','')
    return text

f = open("../tmp/train/car.info.json", encoding='utf-8')
count = 0
final = pd.DataFrame()
for text_json in f.readlines():
    detail = json.loads(text_json)
    config = detail.pop('config')
    param = detail.pop('param')
    count = count + 1
    print(count,'combine',detail['type'])    
    specitems = detail.pop('specitems')
    if len(specitems) == 0:
        continue
    result = pd.DataFrame(specitems)
    result = result[['id']].rename(columns={'id':'specid'})
        
    for i in range(0, len(result)):
        specid = result.loc[i, 'specid']
        
        for j in range(0, len(param)):
            param_temp = re.findall('(?<=\'name\': \').*?(?=\')',str(param[j]))
            param_temp_key = param_temp.pop(0)
            param_temp_name = param_temp
            param_temp_value = re.findall('(?<='+str(specid)+', \'value\': \').*?(?=\')',str(param[j]))
            result.loc[i, param_temp_key] = str(dict(zip(param_temp_name,param_temp_value)))

        for j in range(0, len(config)):
            config_temp = re.findall('(?<=\'name\': \').*?(?=\')',str(config[j]))
            config_temp_key = config_temp.pop(0)
            config_temp_name = config_temp
            config_temp_value = re.findall('(?<='+str(specid)+', \'value\': \').*?(?=\')',str(config[j]))
            result.loc[i, config_temp_key] = str(dict(zip(config_temp_name,config_temp_value)))
    final = final.append(result,sort=False).reset_index(drop=True)
    
final['基本参数'] = final.apply(delete_useless_char,args=('基本参数',),axis=1)
final['车身'] = final.apply(delete_useless_char,args=('车身',),axis=1)
final['发动机'] = final.apply(delete_useless_char,args=('发动机',),axis=1)
final['变速箱'] = final.apply(delete_useless_char,args=('变速箱',),axis=1)
final['底盘转向'] = final.apply(delete_useless_char,args=('底盘转向',),axis=1)
final['车轮制动'] = final.apply(delete_useless_char,args=('车轮制动',),axis=1)
final['主/被动安全装备'] = final.apply(delete_useless_char,args=('主/被动安全装备',),axis=1)
final['辅助/操控配置'] = final.apply(delete_useless_char,args=('辅助/操控配置',),axis=1)
final['外部/防盗配置'] = final.apply(delete_useless_char,args=('外部/防盗配置',),axis=1)
final['内部配置'] = final.apply(delete_useless_char,args=('内部配置',),axis=1)
final['座椅配置'] = final.apply(delete_useless_char,args=('座椅配置',),axis=1)
final['多媒体配置'] = final.apply(delete_useless_char,args=('多媒体配置',),axis=1)
final['灯光配置'] = final.apply(delete_useless_char,args=('灯光配置',),axis=1)
final['玻璃/后视镜'] = final.apply(delete_useless_char,args=('玻璃/后视镜',),axis=1)
final['空调/冰箱'] = final.apply(delete_useless_char,args=('空调/冰箱',),axis=1)
final['电动机'] = final.apply(delete_useless_char,args=('电动机',),axis=1)
final.to_csv('../tmp/train/car_autohome_new_configs.csv', index=False)

## 二.停售车配置信息提取

In [176]:
f = open("../tmp/train/stop.sell.info.json", encoding='utf-8')
count = 0
final = pd.DataFrame()
for text_json in f.readlines():
    detail = json.loads(text_json)
    result = pd.DataFrame([''],columns=['specid'])
    result['specid'] = detail['specid']
    config = detail.pop('result')
    configitems = config['configitems']
    paramitems = config['paramitems']
    
    if (len(paramitems) == 0) | (len(configitems) == 0):
        continue
        
    for i in range(0, len(paramitems)):
        param_temp_name = re.findall('(?<=\'name\': \').*?(?=\')',str(paramitems[i]))
        param_temp_key = re.findall('(?<=\'itemtype\': \').*?(?=\')',str(paramitems[i]))[0]
        param_temp_value = re.findall('(?<=\'value\': \').*?(?=\')',str(paramitems[i]))
        result[param_temp_key] = str(dict(zip(param_temp_name,param_temp_value)))
            
    for i in range(0, len(configitems)):
        config_temp_name = re.findall('(?<=\'name\': \').*?(?=\')',str(configitems[i]))
        config_temp_key = re.findall('(?<=\'itemtype\': \').*?(?=\')',str(configitems[i]))[0]
        config_temp_value = re.findall('(?<=\'value\': \').*?(?=\')',str(configitems[i]))
        result[config_temp_key] = str(dict(zip(config_temp_name,config_temp_value)))
            
    count = count + 1
    print(count,'combine',detail['type'])
    final = final.append(result,sort=False).reset_index(drop=True)

final['基本参数'] = final.apply(delete_useless_char,args=('基本参数',),axis=1)
final['车身'] = final.apply(delete_useless_char,args=('车身',),axis=1)
final['发动机'] = final.apply(delete_useless_char,args=('发动机',),axis=1)
final['变速箱'] = final.apply(delete_useless_char,args=('变速箱',),axis=1)
final['底盘转向'] = final.apply(delete_useless_char,args=('底盘转向',),axis=1)
final['车轮制动'] = final.apply(delete_useless_char,args=('车轮制动',),axis=1)
final['主/被动安全装备'] = final.apply(delete_useless_char,args=('主/被动安全装备',),axis=1)
final['辅助/操控配置'] = final.apply(delete_useless_char,args=('辅助/操控配置',),axis=1)
final['外部/防盗配置'] = final.apply(delete_useless_char,args=('外部/防盗配置',),axis=1)
final['内部配置'] = final.apply(delete_useless_char,args=('内部配置',),axis=1)
final['座椅配置'] = final.apply(delete_useless_char,args=('座椅配置',),axis=1)
final['多媒体配置'] = final.apply(delete_useless_char,args=('多媒体配置',),axis=1)
final['灯光配置'] = final.apply(delete_useless_char,args=('灯光配置',),axis=1)
final['玻璃/后视镜'] = final.apply(delete_useless_char,args=('玻璃/后视镜',),axis=1)
final['空调/冰箱'] = final.apply(delete_useless_char,args=('空调/冰箱',),axis=1)
final['电动机'] = final.apply(delete_useless_char,args=('电动机',),axis=1)
final.to_csv('../tmp/train/car_autohome_stop_configs.csv', index=False)

In [182]:
import pandas as pd
from sqlalchemy import create_engine

ENCODING = 'utf-8'

PRODUCE_DB_ADDR_OUTTER = '101.200.229.249'
PRODUCE_DB_USER = 'pingjia'
PRODUCE_DB_PASSWD = 'De32wsxC'

TEST_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'
engine = create_engine(TEST_PINGJIA_ENGINE, encoding=ENCODING)

with engine.begin() as con:
    sql = 'alter table pingjia.open_cat_detail ADD config_info text Default NULL'
    con.execute(sql)
con.close()

OperationalError: (pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query')
[SQL: alter table pingjia.open_cat_detail ADD config_info text Default NULL]
(Background on this error at: http://sqlalche.me/e/e3q8)